In [1]:
%reload_ext autoreload
%autoreload 2

%gui qt

import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import napari
from pathlib import Path
import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import redCellGUI as rgui

In [3]:
# Immediate next steps for programming work:

# - database development
# - ROICaT index alignment tools


In [16]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [113]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'

# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In session: CR_Hippocannula6/2022-09-05/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!
Computing red cell features... (usually takes 10-20 seconds)
C:\Users\andrew\Documents\localData\CR_Hippocannula6\2022-09-05\701


In [3]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In [3]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [13]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.58 s
Wall time: 512 ms


In [147]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [2]:
fm.getCopyString('ATL027')
fm.s2pTargets('ATL027','2023-08-08','701')

robocopy \\zaru.cortexlab.net\Subjects\ATL027 C:\Users\andrew\Documents\localData\ATL027 /s /xf *.tif
\\zaru.cortexlab.net\Subjects\ATL027\2023-08-08\701
C:\Users\andrew\Documents\localData\ATL027\2023-08-08\701


In [16]:
dbpath = fm.codePath() / 'database'
headerColumns = []
with open(dbpath/'databaseConstruction'/'sessionHeaders.txt') as f:
    for x in f: 
        headerColumns.append(x.strip())

headerColumns

['unique identifier',
 'mouseName',
 'date',
 'session ID',
 'unique mouse ID',
 'experiment type',
 'imaging',
 'face camera',
 'vrEnvironments',
 'number of imaging planes',
 'separation of imaging planes',
 'pockels cell percentage',
 'rotation of objective',
 'rotation of head plate holder',
 'suite2p processed',
 'suite2p quality control',
 'vrExperiment registration processed',
 'red cell quality control',
 'session quality control']

In [37]:
def checkSessionFiles(mouseName, fileIdentifier, onlyTrue=True):
    dataPath = fm.localDataPath()
    sessionPaths = glob(str(dataPath / mouseName) + '/*')
    for spath in sessionPaths:
        cdate = Path(spath).name
        sessions = glob(spath+'/*/')
        for s in sessions:
            cses = Path(s).name
            targetFiles = glob(s+fileIdentifier)
            numFiles = len(targetFiles)
            if not(onlyTrue and numFiles==0): 
                print(f"{cdate}    {cses}    numFiles: {numFiles}")
            
checkSessionFiles('ATL012', '*eye.mj2') #'*eye.mj2' / 'suite2p')

2023-01-30    701    numFiles: 1
2023-02-01    701    numFiles: 1
2023-02-02    701    numFiles: 1
2023-02-06    701    numFiles: 1
2023-02-07    701    numFiles: 1
2023-02-08    702    numFiles: 1
2023-02-09    701    numFiles: 1
2023-02-14    703    numFiles: 1
2023-02-15    701    numFiles: 1
2023-02-16    701    numFiles: 1
2023-02-17    701    numFiles: 1
2023-02-20    701    numFiles: 1
2023-02-21    702    numFiles: 1
2023-02-22    701    numFiles: 1
2023-02-27    701    numFiles: 1
2023-02-28    701    numFiles: 1
2023-03-01    701    numFiles: 1
2023-03-03    701    numFiles: 1
2023-07-10    701    numFiles: 1
2023-07-10    702    numFiles: 1
2023-07-11    701    numFiles: 1
2023-07-13    701    numFiles: 1
2023-07-14    701    numFiles: 1
2023-07-17    701    numFiles: 1
2023-07-18    701    numFiles: 1
2023-07-19    701    numFiles: 1
2023-07-19    702    numFiles: 1
2023-07-19    703    numFiles: 1
2023-07-20    701    numFiles: 1
2023-07-20    702    numFiles: 1


In [39]:
# Note to self -- I'm currently working on migrating my documentation of the data collection over to the access database. 
# I probably want to restart a new "sessions" database and then move everything over from excel. 
# Right now, I'm just moving data from google sheets to excel...

In [15]:
from glob import glob
dataPath = fm.localDataPath()
glob(str(dataPath / 'ATL012')+'/*/*')

['C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-12\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-12\\702',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-12\\703',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-18\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-19\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-20\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-20\\702',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-23\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-24\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-24\\702',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-24\\703',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-25\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-26\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-26\\702',
 'C:\\Users\\andrew\\Documents\\lo

In [11]:
import dbManagement as dbm
vrdb = dbm.vrDatabase()

In [12]:
vrdb.printValues()

(1, 'ATL012', datetime.datetime(2023, 1, 18, 0, 0), 701, 'Blender VR', 1, False, True, True, False, 1, None, 5, 25.0, 6.0, 6.300000190734863, False, True, False, False, False, None, None, 'Very few trials, mouse not trained (for about 6 days) and had it on the wheel an extra 30 minutes...')
(2, 'ATL012', datetime.datetime(2023, 1, 20, 0, 0), 701, 'Blender VR', 1, False, True, True, False, 1, 328.0, 5, 25.0, 8.0, 14.0, False, True, False, False, False, None, None, 'Decent number of trials considering no reward :(')
(3, 'ATL012', datetime.datetime(2023, 1, 20, 0, 0), 702, 'Blender VR', 1, False, True, True, False, 1, 328.0, 5, 25.0, 8.0, 14.0, False, True, False, False, False, None, None, 'Got a surprising number of trials out of it now that reward is working again')
(4, 'ATL012', datetime.datetime(2023, 1, 23, 0, 0), 701, 'Blender VR', 1, False, True, True, False, 1, 328.0, 5, 25.0, 6.0, 14.0, False, True, False, False, False, None, None, 'Mouse sending it, but water was almost dry insi